In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import bootleg_data_prep.benchmarks.filter_and_compute_recall as fcr
import bootleg_data_prep.benchmarks.candidate_generators as gen
import bootleg_data_prep.utils.data_prep_utils as prep_utils
import marisa_trie
import unicodedata
from collections import defaultdict

from nltk.stem import WordNetLemmatizer

import json
import spacy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tag import pos_tag
from tqdm import tqdm

import nltk

ps = PorterStemmer()
nlp = spacy.load("en_core_web_sm")
stopwords = set(stopwords.words('english'))

import bootleg_data_prep.utils.data_prep_utils as prep_utils
from bootleg.utils import utils
from bootleg.symbols.entity_symbols import EntitySymbols
import glob, os, jsonlines, json, time
from tqdm import tqdm
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

In [56]:
TASK_NAME = "RSS"
# TASK_NAME = "KORE"
TASK_NAME

'RSS'

In [57]:
from argparse import Namespace

# args = Namespace(
#     data_dir = '/dfs/scratch0/lorr1/projects/bootleg/data/benchmarks/rss500wikinewdatatest',
#     entity_dump = '/dfs/scratch0/lorr1/projects/bootleg/data/wiki_0906/entity_db/entity_mappings',
#     sub_dir = 'unfiltered',
#     file = "",
#     out_dir = 'filtered',
#     errors_dir = 'cand_gen_filtering_errors',
#     verbose = True,
#     method = "contextual",
#     expand_aliases = True,
#     gold_given = True,
#     max_candidates = 30,
#     large_alias_map = '/dfs/scratch0/lorr1/data_prep/unfiltered_data_0906/entity_db/entity_mappings/alias2qids.json',
#     large_title_map = '/dfs/scratch0/lorr1/data_prep/unfiltered_data_0906/entity_db/entity_mappings/qid2title.json',
#     wiki_pages = '/dfs/scratch0/lorr1/data_prep/unfiltered_data_0914/'
# )
if TASK_NAME == 'RSS':
    args = Namespace(
#         data_dir = '/dfs/scratch1/senwu/mmtl/test/bootleg/bootleg-internal/data/benchmarks/rss500wikinewdatatest',
#         entity_dump = '/dfs/scratch1/senwu/mmtl/test/bootleg/bootleg-internal/data/wiki_0906/entity_db/entity_mappings',
        data_dir = '/dfs/scratch0/lorr1/projects/bootleg/data/benchmarks/rss500wikinewdatatest',
        entity_dump = '/dfs/scratch0/lorr1/projects/bootleg/data/wiki_0906/entity_db/entity_mappings',
        sub_dir = 'unfiltered',
        file = "",
        out_dir = 'filtered',
        errors_dir = 'cand_gen_filtering_errors',
        verbose = True,
        method = "contextual",
        expand_aliases = True,
        gold_given = True,
        max_candidates = 30,
        large_alias_map = '/dfs/scratch1/senwu/mmtl/test/bootleg/bootleg-internal/data/wiki_0906/entity_db/entity_mappings/alias2qids_modified_name_v6.json',
    #     large_alias_map = '/dfs/scratch0/lorr1/data_prep/unfiltered_data_0906/entity_db/entity_mappings/alias2qids.json',
        large_title_map = '/dfs/scratch0/lorr1/data_prep/unfiltered_data_0906/entity_db/entity_mappings/qid2title.json',
        wiki_pages = '/dfs/scratch0/lorr1/data_prep/unfiltered_data_0914/'
    )

In [58]:
if TASK_NAME == 'KORE':
    args = Namespace(
        data_dir = '/dfs/scratch0/lorr1/projects/bootleg/data/benchmarks/kore50wikinewdatatest',
        entity_dump = '/dfs/scratch0/lorr1/projects/bootleg/data/wiki_0906/entity_db/entity_mappings',
#         data_dir = '/dfs/scratch1/senwu/mmtl/test/bootleg/bootleg-internal/data/benchmarks/kore50wikinewdatatest',
#         entity_dump = '/dfs/scratch1/senwu/mmtl/test/bootleg/bootleg-internal/data/wiki_0906/entity_db/entity_mappings',
        sub_dir = 'unfiltered',
        file = "",
        out_dir = 'filtered',
        errors_dir = 'cand_gen_filtering_errors',
        verbose = True,
        method = "contextual",
        expand_aliases = True,
        gold_given = True,
        max_candidates = 30,
        large_alias_map = '/dfs/scratch1/senwu/mmtl/test/bootleg/bootleg-internal/data/wiki_0906/entity_db/entity_mappings/alias2qids_modified_name_v6.json',
    #     large_alias_map = '/dfs/scratch0/lorr1/data_prep/unfiltered_data_0906/entity_db/entity_mappings/alias2qids.json',
        large_title_map = '/dfs/scratch0/lorr1/data_prep/unfiltered_data_0906/entity_db/entity_mappings/qid2title.json',
        wiki_pages = '/dfs/scratch0/lorr1/data_prep/unfiltered_data_0914/'
    )
    
# args = Namespace(
#     data_dir = '/dfs/scratch0/lorr1/projects/bootleg/data/benchmarks/kore50wikinewdatatest',
#     entity_dump = '/dfs/scratch0/lorr1/projects/bootleg/data/wiki_0906/entity_db/entity_mappings',
#     sub_dir = 'unfiltered',
#     file = "",
#     out_dir = 'filtered',
#     errors_dir = 'cand_gen_filtering_errors',
#     verbose = True,
#     method = "contextual",
#     expand_aliases = True,
#     gold_given = True,
#     max_candidates = 30,
#     large_alias_map = '/dfs/scratch0/lorr1/data_prep/unfiltered_data_0906/entity_db/entity_mappings/alias2qids.json',
#     large_title_map = '/dfs/scratch0/lorr1/data_prep/unfiltered_data_0906/entity_db/entity_mappings/qid2title.json',
#     wiki_pages = '/dfs/scratch0/lorr1/data_prep/unfiltered_data_0914/'
# )

In [6]:
def load_wikipedia_pages(args):
    files = sorted(glob.glob(os.path.join(args.wiki_pages, '*.jsonl')), reverse=True)
    print(f"Loading {len(files)} wikipedia pages...")
    st = time.time()
    qid2page = {}
    for file in tqdm(files):
        with jsonlines.open(file) as in_file:
            for obj in in_file:
                qid2page[obj['qid']] = obj
    print(f"Took {time.time()-st}s to load {len(files)} files")
    return qid2page

def load_large_alias_map(args, entity_dump):
    temp_dir = "_temp"
    utils.ensure_dir(temp_dir)
    temp_f = os.path.join(temp_dir, "alias_map.pkl")
#     if os.path.exists(temp_f):
#         print(f"Reading alias map from {temp_f}")
#         filtered_alias_map = utils.load_pickle_file(temp_f)
#         return filtered_alias_map
    large_alias_to_qid_map = json.load(open(args.large_alias_map))
#     print(large_alias_to_qid_map['richard bernstein'])
    # filter down and remove qids which aren't contained in the entity save
    filtered_alias_map = {}
    for alias, qid_list in large_alias_to_qid_map.items():
        filtered_qid_list = []
        for qid, count in qid_list:
            if entity_dump.qid_exists(qid):
                filtered_qid_list.append([qid, count])
        filtered_alias_map[alias] = filtered_qid_list
    utils.dump_pickle_file(temp_f, filtered_alias_map)
    return filtered_alias_map

def load_large_title_map(args):
    return json.load(open(args.large_title_map))

def write_output_file(sentences, fpath, args):
    print(f"Output to {fpath}")
    out_file = open(fpath, 'w')
    for i, sentence in enumerate(sentences):
        out_file.write(json.dumps(sentence) + "\n")

In [7]:
qid2page = load_wikipedia_pages(args)

  0%|          | 0/241 [00:00<?, ?it/s]

Loading 241 wikipedia pages...


100%|██████████| 241/241 [35:15<00:00,  8.78s/it]  

Took 2115.7973625659943s to load 241 files


In [8]:
entity_dump = EntitySymbols(load_dir=args.entity_dump)
alias_map = load_large_alias_map(args, entity_dump)
title_map = load_large_title_map(args)
alias_tri = marisa_trie.Trie(alias_map.keys())

In [9]:
# alias_map = load_large_alias_map(args, entity_dump)
# alias_tri = marisa_trie.Trie(alias_map.keys())
# print(alias_map['richard bernstein'])
# print(alias_map['national marriage project'])

In [10]:
def spacy_to_split_remapping(tokens, spacy_tokens):
    norm_cur = 0
    spacy_cur = 0
    spacy_remap = [0] * len(spacy_tokens)
    spacy_tokens = [str(token) for token in spacy_tokens]
    cur_str = ""

    while norm_cur < len(tokens) and spacy_cur < len(spacy_tokens):
        if tokens[norm_cur] == spacy_tokens[spacy_cur]:
            spacy_remap[spacy_cur] = norm_cur
            spacy_cur+=1
            norm_cur+=1
        else:
            if tokens[norm_cur].startswith(cur_str + spacy_tokens[spacy_cur]):
                if tokens[norm_cur] == cur_str + spacy_tokens[spacy_cur]:
                    spacy_remap[spacy_cur] = norm_cur
                    spacy_cur+=1
                    norm_cur+=1
                    cur_str = ""
                else:
                    spacy_remap[spacy_cur] = norm_cur
                    cur_str += spacy_tokens[spacy_cur]
                    spacy_cur+=1
    return spacy_remap

In [88]:
class Contextual:

    def __init__(self, args, entity_dump, qid2page, alias_map, alias_tri, title_map):
        self.temp = 0
        self.args = args
        self.entity_dump = entity_dump
        self.filtered_alias_map = alias_map
        self.title_map = title_map
        self.alias_tri = alias_tri
        self.qid2page = qid2page
        self.new_alias2qid = {}
        self.saved_metrics = defaultdict(int)
        self.detailed_metrics = defaultdict(list)

    def convert_char_to_word_boundaries(self, phrase, start_char, end_char):
        start_word = max(0, len(phrase[:start_char].split()))
        end_word = max(0, len(phrase[:end_char].split()))
        return start_word, end_word

    def get_lnrm(self, s):
        """Convert a string to its lnrm form
        We form the lower-cased normalized version l(s) of a string s by canonicalizing
        its UTF-8 characters, eliminating diacritics, lower-casing the UTF-8 and
        throwing out all ASCII-range characters that are not alpha-numeric.
        from http://nlp.stanford.edu/pubs/subctackbp.pdf Section 2.3
        Args:
            input string
        Returns:
            the lnrm form of the string
        """
        lnrm = unicodedata.normalize('NFD', str(s))
        lnrm = lnrm.lower()
        lnrm = lnrm.replace("-", " ")
        lnrm = ''.join([x for x in lnrm if (not unicodedata.combining(x)
                                            and x.isalnum() or x == ' ')]).strip()
        # will remove if there are any duplicate white spaces e.g. "the  alias    is here"
        lnrm = " ".join(lnrm.split())
        return lnrm

    def get_proper_nouns(self, text):
        words = text.split()
        tagged_text = pos_tag(words)
        # all_nouns = [word for word,pos in tagged_text if pos[:2] in ['NN'] and (word.lower() not in stopwords) and (word.lower().rstrip("s") not in stopwords)]
        all_nouns = []
        #
        # doc = nlp(text)
        # ent_idx = 0
        # while ent_idx < len(doc.ents):
        #     ent = doc.ents[ent_idx]
        #     ent_text = ent.text
        #     ent_idx += 1
        #     print("ENT TX", ent_text)
        i = 0
        nouns = []
        while i < len(words):
            if words[i][0].isupper() and words[i].lower() not in stopwords:
                start_i = i
                i += 1
                while True and i < len(words):
                    if words[i][0].isupper():
                        i += 1
                    else:
                        break
                nouns.append(' '.join(words[start_i: i]))
                continue
            i += 1
        full_nouns = set(all_nouns)
        for noun in nouns:
            full_nouns.add(noun)
            for part in noun.split():
                full_nouns.add(part)
        return list(full_nouns)

    def get_proper_nouns_old(self, text):
        nouns = []
        words = text.split()
        start = 0
        i = 0
        while i < len(words):
            if words[i][0].isupper():
                start_i = i
                i += 1
                while True and i < len(words):
                    if words[i][0].isupper():
                        i += 1
                    else:
                        break
                nouns.append(' '.join(words[start_i: i]))
                continue
            i += 1
        full_nouns = []
        for noun in nouns:
            full_nouns.append(noun)
            for part in noun.split():
                full_nouns.append(part)
        return list(set(full_nouns))

    def get_sub_alias_match(self, alias, alias2qids):
        assert len(alias.strip()) > 0, f"The alias is empty. This shouldn't happen."
        words = alias.split()
        subseqs = set()
        for i in range(len(words)):
            for j in range(i, len(words)+1):
                phrase = ' '.join(words[i:j])
                if len(phrase) > 0:
                    subseqs.add(phrase)
        subseqs = list(subseqs)
        subseqs.sort(key=lambda s: len(s), reverse=True)
        qid2count = defaultdict(int)
        curr_len = len(subseqs[0].split())
        for alias in subseqs:
            if len(alias.split()) != curr_len:
                if len(qid2count) > 0:
                    break
                curr_len = len(alias.split())
            if alias in alias2qids:
                for pair in alias2qids[alias]:
                    qid2count[pair[0]] = pair[1] + qid2count[pair[0]]
        candidates = [[qid, value] for qid, value in qid2count.items() if value > 0]
        return candidates

    def get_sub_alias_match2(self, alias, alias2qids):
        wnl = WordNetLemmatizer()
        assert len(alias.strip()) > 0, f"The alias is empty. This shouldn't happen."
        words = alias.split()
        subseqs = set()
        if wnl.lemmatize(words[-1]) != words[-1]:
            subseqs.add(' '.join(words[:-1] + [wnl.lemmatize(words[-1])]))
        for i in range(len(words)):
            for j in range(i, len(words)+1):
                phrase = ' '.join(words[i:j])
                if len(phrase) > 0 and len(phrase.split()) > 0:
                    subseqs.add(phrase)
        if len(subseqs) <= 0:
            return []
        subseqs = list(subseqs)
        subseqs.sort(key=lambda s: len(s), reverse=True)
        qid2count = defaultdict(int)
        curr_len = len(subseqs[0].split())
        for alias in subseqs:
            if len(alias.split()) != curr_len:
                if len(qid2count) > 0:
                    break
                curr_len = len(alias.split())
            if alias in alias2qids:
                for pair in alias2qids[alias]:
                    qid2count[pair[0]] = pair[1] + qid2count[pair[0]]
        candidates = [[qid, value] for qid, value in qid2count.items() if value > 0]
        return candidates

    def get_qids(self, target_alias, alias2qids):       
        if target_alias in alias2qids:
            return alias2qids[target_alias]

        if target_alias.replace(" ", "") in alias2qids:
            return alias2qids[target_alias.replace(" ", "")]

        return self.get_sub_alias_match2(target_alias, alias2qids)


    def recompute_qid_rank(self, sample, alias, true_qid, qid2page, alias2qid, entity_dump):
        qid_pairs = self.get_qids(alias, alias2qid)
        if len(qid_pairs) == 0:
            self.saved_metrics["no_alias"] += 1
            self.detailed_metrics["no_alias"].append(
                {
                    "true_qid": true_qid,
                    "alias": alias
                }
            )
        if len(qid_pairs) <= 1:
            self.saved_metrics["len_1_alias"] += 1
            return qid_pairs
        # Each element in qid_pairs is [qid, qid_score]
        qid_pairs.sort(key=lambda x:x[1], reverse=True)
        context = sample['sentence']
        prop_nouns = [word.lower() for word in self.get_proper_nouns(context) if word.lower() != alias]
        qids = [x[0] for x in qid_pairs]
        saved_counts = {}
        for i, qid in enumerate(qids):
            count = {p: 0 for p in prop_nouns}
            if qid in qid2page:
                for sentence in qid2page[qid]['sentences']:
                    text = " " + sentence['sentence'].lower() + " "
                    for term in prop_nouns:
                        if f" {term} " in text:
                            count[term] += 1
            saved_counts[qid] = count
        counts = [x[1] for x in qid_pairs]
        # Compute score for qid based on counts of proper nouns
        for i, qid in enumerate(qids):
            score = 0
            num_hits = 0
            for w in list(saved_counts[qid].keys()):
                x = saved_counts[qid][w]
                if x > 0:
                    num_hits += 1
                    score += min(x, 5)*len(w.split())
            counts[i] += 10000*score
        # TODO: fix if acronym
#         print(counts)
#         ddd =[]
#         eee = []
#         for i, qid in enumerate(qids):
#             title = self.get_lnrm(entity_dump.get_title(qid).lower())
#             title_score = fuzz.partial_ratio(title, alias)/100
#             title_score2 = nltk.jaccard_distance(set(alias.split()), set(title.split()))
#             ddd.append(100000*title_score)
#             eee.append(100*title_score2)
#         print(ddd)
#         print(eee)

        for i, qid in enumerate(qids):
            title = self.get_lnrm(entity_dump.get_title(qid).lower())
            title_score = fuzz.partial_ratio(title, alias)/100
            counts[i] += 100000*title_score
            title_score2 = 1 - nltk.jaccard_distance(set(alias.split()), set(title.split()))
#             title_score2 = fuzz.ratio(title, alias)/100
            counts[i] += 10000*title_score2
#         print(counts)
#         print([self.get_lnrm(entity_dump.get_title(qid).lower()) for i, qid in enumerate(qids)])


        scored_qids = [list(x) for x in zip(qids, counts)]
        scored_qids = sorted(scored_qids, key= lambda x: x[1], reverse = True)
        # Take top 5 most popular qids and remaining 25 from scored qids
        final_scored_qids = qid_pairs[:5]
        only_qids = [x[0] for x in final_scored_qids]
        for qid_pair in scored_qids:
            if len(final_scored_qids) == self.args.max_candidates:
                break
            if qid_pair[0] not in only_qids:
                only_qids.append(qid_pair[0])
                final_scored_qids.append(qid_pair)
        if self.args.gold_given:
            if true_qid not in only_qids[:self.args.max_candidates]:
                if true_qid in [x[0] for x in scored_qids]:
                    self.saved_metrics["in_long_cands_not_short"] += 1
                    self.detailed_metrics["in_long_cands_not_sort"].append(
                        {
                            "true_qid": true_qid,
                            "alias": alias,
                            "context": context,
                            "nouns": prop_nouns,
                            "saved_counts": saved_counts,
                            "scored_qids": scored_qids,
                            "titles": [entity_dump.get_title(q) for q in only_qids]
                        }
                    )
#                     print("TRUE QID", true_qid, entity_dump.get_title(true_qid))
#                     print("CONTEXT", context)
#                     print("NOUNS", prop_nouns)
#                     print("COUNTS", saved_counts[true_qid])
#                     for tqid in only_qids[:self.args.max_candidates]:
#                         print("OTHER COUNTS", tqid, entity_dump.get_title(tqid),  saved_counts[tqid])
#                     print("SCORES", scored_qids[:50])
#                     print("TITLES", [entity_dump.get_title(q) for q in only_qids[:50]])
                    # print("ALL SENTENCES", qid2page[true_qid]['sentences'])
                else:
                    self.saved_metrics["not_long_cands_not_short"] += 1
                    self.detailed_metrics["not_long_cands_not_sort"].append(
                        {
                            "true_qid": true_qid,
                            "alias": alias,
                            "context": context,
                            "nouns": prop_nouns,
                            "saved_counts": saved_counts,
                            "scored_qids": scored_qids,
                            "titles": [entity_dump.get_title(q) for q in only_qids]
                        }
                    )
            else:
                self.saved_metrics["success"] += 1
        return final_scored_qids[:self.args.max_candidates]

    def whitespace_tokenize(self, text):
        """Runs basic whitespace cleaning and splitting on a piece of text."""
        text = text.strip()
        if not text:
            return []
        tokens = text.split()
        return tokens
    
    def filter_sentence(self, sentence):
        print()
        print()
        print(sentence['sent_idx_unq'])
#         print(sentence)
        total_cnt = 0
        success = 0
        no_alias_exists = 0
        qid_not_in_cands = 0
        no_qid_in_dump = 0
        errors = []
        new_sentence = {
            'sentence': sentence['sentence'],
            'sent_idx_unq': sentence['sent_idx_unq'],
            'aliases': [],
            'spans': [],
            'qids': [],
            'gold': []
        }

        phrase = sentence['sentence']
        aliases = sentence['aliases']
        spans = sentence['spans']
        if self.args.expand_aliases:
#             print("PRINTING ALL QIDS:\n")
#             for i in range(len(aliases)):
#                 if sentence['qids'][i] in self.title_map:
#                     print(sentence['qids'][i], self.title_map[sentence['qids'][i]], sentence['spans'][i], sentence['sentence'].split()[sentence['spans'][i][0]: sentence['spans'][i][1]])
#                 else:
#                     print(sentence['qids'][i], "None", sentence['spans'][i], sentence['sentence'].split()[sentence['spans'][i][0]: sentence['spans'][i][1]])
#             print()
            
            doc = nlp(phrase)
            split_tokens = self.whitespace_tokenize(phrase)

            print("SANITY CHECK ALL ALIASES:\n")
            for i in range(len(aliases)):
                print("CHECKING>>>", aliases[i].replace(' ', '') in phrase.replace(' ', '').lower(), phrase.replace(' ', '').lower().count(aliases[i].replace(' ', '')))

            for i in range(len(aliases)):
                if aliases[i].lower() != " ".join(split_tokens[spans[i][0]:spans[i][1]]).lower():
                    print("WARNING PREV>>>", aliases[i] in phrase.lower(), aliases[i].replace(' ', '') in phrase.replace(' ', '').lower(), aliases[i], split_tokens[spans[i][0]:spans[i][1]])
                
            for i in range(len(aliases)):
                if aliases[i].lower() != " ".join(split_tokens[spans[i][0]:spans[i][1]]).lower():
                    st = spans[i][0]
                    ed = spans[i][1]
                    while st < len(split_tokens) and not split_tokens[st].lower().replace(' ', '').startswith(aliases[i].lower().replace(' ', '')) and not aliases[i].lower().replace(' ', '').startswith(split_tokens[st].lower().replace(' ', '')):
                        st += 1
                    while st > 0 and not split_tokens[st].lower().replace(' ', '').startswith(aliases[i].lower().replace(' ', '')) and not aliases[i].lower().replace(' ', '').startswith(split_tokens[st].lower().replace(' ', '')):
                        st -= 1
                    if ed < st + 1: ed = st + 1
                    while " ".join(split_tokens[st:ed]).lower().replace(' ', '').startswith(aliases[i].lower().replace(' ', '')) and " ".join(split_tokens[st:ed]).lower().replace(' ', '') != aliases[i].lower().replace(' ', ''):
                        ed -= 1
                    while aliases[i].lower().replace(' ', '').startswith(" ".join(split_tokens[st:ed]).lower().replace(' ', '')) and " ".join(split_tokens[st:ed]).lower().replace(' ', '') != aliases[i].lower().replace(' ', ''):
                        ed += 1
                    spans[i] = [st, ed]
                    
            print("SPAN CORRECTION:\n")
            for i in range(len(aliases)):
                print("CHECKING>>>", aliases[i].replace(' ', '') in phrase.replace(' ', '').lower(), phrase.replace(' ', '').lower().count(aliases[i].replace(' ', '')))

            for i in range(len(aliases)):
                if aliases[i].lower() != " ".join(split_tokens[spans[i][0]:spans[i][1]]).lower():
                    print("WARNING MID>>>", aliases[i] in phrase.lower(), aliases[i].replace(' ', '') in phrase.replace(' ', '').lower(), aliases[i], split_tokens[spans[i][0]:spans[i][1]])

                    
            spacy_tokens = [x for x in doc]
            spacy_to_split_remap = spacy_to_split_remapping(split_tokens, spacy_tokens)
#             print(spacy_to_split_remap)

#             print([(X.text, X.label_, X.start, X.end, split_tokens[spacy_to_split_remap[X.start]:spacy_to_split_remap[X.end]]) for X in doc.ents])
            ner_phrases = []
            for X in doc.ents:
                if spacy_to_split_remap[X.start] == spacy_to_split_remap[X.end]:
                    ner_phrases.append((spacy_to_split_remap[X.start],spacy_to_split_remap[X.end]+1, X.label_))
                else:
                    ner_phrases.append((spacy_to_split_remap[X.start],spacy_to_split_remap[X.end], X.label_))

            flag = False
            for i in range(len(aliases)):
                if (sentence['spans'][i][0], sentence['spans'][i][1]) not in ner_phrases:
                    if sentence['qids'][i] in self.title_map:
                        if self.title_map[sentence['qids'][i]].lower() == ' '.join(sentence['sentence'].split()[sentence['spans'][i][0]: sentence['spans'][i][1]]).lower():
                            continue
                        flag = True
                    else:
                        flag = True
            if flag: print(sentence)

            for i in range(len(aliases)):
                if (sentence['spans'][i][0], sentence['spans'][i][1]) not in ner_phrases:
                    if sentence['qids'][i] in self.title_map:
                        if self.title_map[sentence['qids'][i]].lower() == ' '.join(sentence['sentence'].split()[sentence['spans'][i][0]: sentence['spans'][i][1]]).lower():
                            continue
#                         print(sentence)
#                         print(sentence['qids'][i], self.title_map[sentence['qids'][i]], sentence['spans'][i], sentence['sentence'].split()[sentence['spans'][i][0]: sentence['spans'][i][1]])
#                         print([(X.text, X.label_, X.start, X.end, split_tokens[spacy_to_split_remap[X.start]:spacy_to_split_remap[X.end]]) for X in doc.ents])
                        total_cnt += 1
                    else:
#                         print(sentence)
#                         print(sentence['qids'][i], "None", sentence['spans'][i], sentence['sentence'].split()[sentence['spans'][i][0]: sentence['spans'][i][1]])
#                         print([(X.text, X.label_, X.start, X.end, split_tokens[spacy_to_split_remap[X.start]:spacy_to_split_remap[X.end]]) for X in doc.ents])
                        total_cnt += 1
#             print(spans, aliases)
            import copy
            ori_aliases = copy.deepcopy(aliases)

#             prefix = ['the', "``", "-lrb-", "-rrb-"]
#             suffix = ["'", "'s", "-lrb-", "-rrb-"]
            for i in range(len(aliases)):
                if (sentence['spans'][i][0], sentence['spans'][i][1]) in ner_phrases: continue
                for j in ner_phrases:
                    if j[0] <= sentence['spans'][i][0] and sentence['spans'][i][1] <= j[1]:
                        st = j[0]
                        ed = j[1]
                        check = False
                        for k in range(len(aliases)):
                            if i == k: continue
                            if len(set(list(range(st, ed))).intersection(set(list(range(sentence['spans'][k][0], sentence['spans'][k][1]))))) > 0:
                                check = True
                                break
                        if check: break
#                         if split_tokens[st].lower() in prefix: st += 1
#                         if ed-1>=0 and split_tokens[ed-1].lower() in suffix: ed -= 1
                        aliases[i] = " ".join(split_tokens[st:ed])
                        spans[i] = [st, ed]

            prefix = ['the', "``", "-lrb-", "-rrb-"]
            suffix = ["'", "'s", "-lrb-", "-rrb-"]
            for i in range(len(aliases)):
#                 if (sentence['spans'][i][0], sentence['spans'][i][1]) in ner_phrases: continue
#                 for j in ner_phrases:
#                     if j[0] <= sentence['spans'][i][0] and sentence['spans'][i][1] <= j[1]:
                st = spans[i][0]
                ed = spans[i][1]
                if split_tokens[st].lower() in prefix: st += 1
                if ed-1>=0 and split_tokens[ed-1].lower() in suffix: ed -= 1
                aliases[i] = " ".join(split_tokens[st:ed])
                spans[i] = [st, ed]


#             print(">>>>>>>>>>>>>", [x.lower() for x in ori_aliases] == [x.lower() for x in aliases], ori_aliases, aliases)
#             print(split_tokens)
#             print(ner_phrases)
            suf_comp = ["of", "'s"]
            pre_comp = ["of", "'s"]
            suf_comp = ["'s"]
#             suf_comp = []    
            for i in range(len(spans)):
                if split_tokens[spans[i][1]].lower() in suf_comp and spans[i][1] + 1 in [x[0] for x in ner_phrases]:
#                     print("@@@@@@@")
#                     print("ORIGINAL", aliases[i])
                    st = spans[i][0]
                    ed = spans[i][1]
                    for ner_phrase in ner_phrases:
                        if ner_phrase[0] == ed + 1 and ner_phrase[2] != 'GPE':
                            ed = ner_phrase[1]
                            break
                    aliases[i] = " ".join(split_tokens[st:ed])
                    spans[i] = [st, ed]
#                     print("UPDATED", aliases[i])
                if spans[i][1] in [x[0] for x in ner_phrases]:
#                     print("@@@@@@@")
#                     print("ORIGINAL", aliases[i])
                    st = spans[i][0]
                    ed = spans[i][1]
                    for ner_phrase in ner_phrases:
                        if ner_phrase[0] == ed:
                            ed = ner_phrase[1]
                            break
                    aliases[i] = " ".join(split_tokens[st:ed])
                    spans[i] = [st, ed]
#                     print("UPDATED", aliases[i])
            for i in range(len(spans)):
                if spans[i][0] > 0 and split_tokens[spans[i][0] - 1].lower() in pre_comp and spans[i][0] - 1 in [x[1] for x in ner_phrases]:
#                     print("@@@@@@@")
#                     print("ORIGINAL", aliases[i])
                    st = spans[i][0]
                    ed = spans[i][1]
                    for ner_phrase in ner_phrases:
                        if ner_phrase[1] == st - 1 and ner_phrase[2] == 'ORG':
                            st = ner_phrase[0]
                            break
                    aliases[i] = " ".join(split_tokens[st:ed])
                    spans[i] = [st, ed]
#                     print("UPDATED", aliases[i])
                if spans[i][0] in [x[1] for x in ner_phrases]:
#                     print("@@@@@@@")
#                     print("ORIGINAL", aliases[i])
                    st = spans[i][0]
                    ed = spans[i][1]
                    for ner_phrase in ner_phrases:
                        if ner_phrase[0] == st:
                            st = ner_phrase[0]
                            break
                    aliases[i] = " ".join(split_tokens[st:ed])
                    spans[i] = [st, ed]
#                     print("UPDATED", aliases[i])

                    
#             print("UPDATING..........")
            for i in range(len(aliases)):
                if (sentence['spans'][i][0], sentence['spans'][i][1]) not in ner_phrases:
                    if sentence['qids'][i] in self.title_map:
                        if self.title_map[sentence['qids'][i]].lower() == ' '.join(sentence['sentence'].split()[sentence['spans'][i][0]: sentence['spans'][i][1]]).lower():
                            continue
#                         print(sentence)
#                         print(sentence['qids'][i], self.title_map[sentence['qids'][i]], sentence['spans'][i], sentence['sentence'].split()[sentence['spans'][i][0]: sentence['spans'][i][1]])
#                         print([(X.text, X.label_, X.start, X.end, split_tokens[spacy_to_split_remap[X.start]:spacy_to_split_remap[X.end]]) for X in doc.ents])
                        total_cnt += 1
                    else:
#                         print(sentence)
#                         print(sentence['qids'][i], "None", sentence['spans'][i], sentence['sentence'].split()[sentence['spans'][i][0]: sentence['spans'][i][1]])
#                         print([(X.text, X.label_, X.start, X.end, split_tokens[spacy_to_split_remap[X.start]:spacy_to_split_remap[X.end]]) for X in doc.ents])
                        total_cnt += 1

#             print(spans, aliases)

#             print()
#             print()
#             print(sentence['sentence'])
#             for i in range(len(spans)):
#                 if sentence['qids'][i] in self.title_map and aliases[i].lower() == self.title_map[sentence['qids'][i]].lower(): continue
#                 if sentence['qids'][i] in self.title_map:
#                     print("!!!!!!!!!!!!", sentence['qids'][i], self.title_map[sentence['qids'][i]], "_____", aliases[i])

#             for i in range(len(aliases)):
#                 if aliases[i].lower() != " ".join(split_tokens[spans[i][0]:spans[i][1]]).lower() or (sentence['qids'][i] in self.title_map and aliases[i].lower() != self.title_map[sentence['qids'][i]].lower()):
#                     print("WARNING AFTER>>>", aliases[i] in phrase.lower(), aliases[i].replace(' ', '') in phrase.replace(' ', '').lower(), aliases[i], split_tokens[spans[i][0]:spans[i][1]], self.title_map[sentence['qids'][i]])
            

#             al_idx = 0
#             ent_idx = 0
#             while ent_idx < len(doc.ents) and al_idx < len(aliases):
#                 ent = doc.ents[ent_idx]
#                 ent_text = ent.text
#                 ent_start_char = ent.start_char
#                 ent_end_char = ent.end_char
#                 # remove starting "the" and trailing "'s"
#                 if ent_text.startswith("the "):
#                     ent_text = ent_text[4:]
#                     ent_start_char += 4
#                 if ent_text.endswith(" 's"):
#                     ent_text = ent_text[:-3]
#                     ent_end_char -= 3
#                 start_span, end_span = self.convert_char_to_word_boundaries(phrase, ent_start_char, ent_end_char)
#                 cur_alias = aliases[al_idx]
#                 cur_spans = spans[al_idx]
#                 is_bleeding_into_next = False
#                 if al_idx+1 < len(spans):
#                     next_cur_start_span = spans[al_idx+1][0]
#                     is_bleeding_into_next = end_span >= next_cur_start_span
#                 if start_span >= cur_spans[1]:
#                     al_idx += 1
#                 elif start_span <= cur_spans[0] and end_span >= cur_spans[1] and not is_bleeding_into_next and aliases[al_idx] != ent_text.lower():
# #                     print("Swapping", start_span, end_span, cur_spans, cur_alias, "**", ent_text, "**", phrase)
#                     spans[al_idx] = [start_span, end_span]
#                     aliases[al_idx] = ent_text
#                     al_idx += 1
#                     ent_idx += 1
#                 else:
#                     ent_idx += 1
        if self.args.gold_given:
            qids = sentence['qids']
        else:
            qids = ["Q-1" for _ in range(len(sentence['aliases']))]
        gold = sentence.get('gold', [True for _ in range(len(sentence['aliases']))])
        sent_idx_unq = sentence['sent_idx_unq']
        new_sentence['cands'] = []
        for i in range(len(aliases)):
            self.temp += 1
            alias = self.get_lnrm(aliases[i])
#             alias = prep_utils.get_lnrm(aliases[i])
            candidates = self.recompute_qid_rank(sentence, alias, qids[i], self.qid2page, self.filtered_alias_map, self.entity_dump)
            new_sentence['cands'].append(candidates)
            # With no golds and not nil model, need to set QIDs to be something in the save; nil models can have Q-1
            qid_to_add = qids[i]
#             if not self.args.gold_given and len(candidates) > 0:
#                 qid_to_add = candidates[0][0]
#             print(f"Alias: {alias}, Candidates {candidates}")
            if len(candidates) == 0:
                print(f"ERROR {alias} not in alias map. True QID: {qids[i]}")
                errors.append({
                    'alias': alias,
                    'qid': qid_to_add,
                    'sentence': sentence['sentence'],
                    'label': 'alias not found',
                })
                no_alias_exists += 1
                continue
            if self.args.gold_given and not self.entity_dump.qid_exists(qid_to_add):
                print(f"ERROR QID {qid_to_add} doesn't exist in entity save for {alias}.")
                no_qid_in_dump += 1
                continue
            if qid_to_add in [x[0] for x in candidates]:
                assert int(spans[i][1]) <= len(sentence['sentence'].split()), sentence
                alias = f"{alias}_{sent_idx_unq}"
                self.new_alias2qid[alias] = candidates
                new_sentence['aliases'].append(alias)
                new_sentence['qids'].append(qid_to_add)
                new_sentence['spans'].append(spans[i])
                new_sentence['gold'].append(gold[i])
                success += 1
            else:
                # print(f"Alias: {alias}, True QID: {qids[i]}, Candidates in map: {candidates}")
                errors.append({
                    'alias': alias,
                    'candidates': list(candidates),
                    'qid': qid_to_add,
                    'sentence': sentence['sentence'],
                    'label': 'candidate not found',
                })
                assert int(spans[i][1]) <= len(sentence['sentence'].split()), sentence
                alias = f"{alias}_{sent_idx_unq}"
                self.new_alias2qid[alias] = candidates
                new_sentence['aliases'].append(alias)
                new_sentence['qids'].append(qid_to_add)
                new_sentence['spans'].append(spans[i])
                new_sentence['gold'].append(gold[i])
                qid_not_in_cands += 1
                
        
        if len(new_sentence['aliases']) != len(ori_aliases):
            print("FINAL", ori_aliases, new_sentence['aliases'])
#         print(total_cnt)
        # return number of aliases kept, and dropped
        return new_sentence, success, no_alias_exists, qid_not_in_cands, no_qid_in_dump, errors

In [89]:
out_dir = prep_utils.get_outdir(args.data_dir, args.out_dir)
errors_dir = prep_utils.get_outdir(args.data_dir, args.errors_dir)

print(f"Candidate generator: {args.method}")
if args.method == 'standard':
    candidate_generator = gen.Standard(args, entity_dump)
elif args.method == 'contextual':
    candidate_generator = Contextual(args, entity_dump, qid2page, alias_map, alias_tri, title_map)

Making /dfs/scratch0/lorr1/projects/bootleg/data/benchmarks/rss500wikinewdatatest/filtered
Making /dfs/scratch0/lorr1/projects/bootleg/data/benchmarks/rss500wikinewdatatest/cand_gen_filtering_errors
Candidate generator: contextual


In [ ]:
candidate_generator.saved_metrics = defaultdict(int)
candidate_generator.detailed_metrics = defaultdict(list)
if args.file:
    files = [args.file]
else:
    files = prep_utils.glob_files(os.path.join(args.data_dir, args.sub_dir, '*.jsonl'))
recall_results = {}
print("FILES", files)
for f in files:
    print(f"Filtering file {f}")
    fname = os.path.basename(f)
    if fname == "test_rss500_clean.jsonl":
        print(f"Skipping {fname}")
        continue
    filtered_sentences, errors, recall = fcr.filter_and_clean(f, candidate_generator, args)
    write_output_file(filtered_sentences, os.path.join(out_dir, fname), args)
    fcr.write_errors(errors, os.path.join(errors_dir, fname.replace('jsonl', 'txt')))
    recall_results[fname] = recall

entity_dump_dir = os.path.join(out_dir, 'entity_db/entity_mappings')
utils.ensure_dir(entity_dump_dir)
new_alias_f = os.path.join(entity_dump_dir, "alias2qid_new.json")
# save new alias mapping
with open(new_alias_f, "w") as out_f:
    json.save(candidate_generator.new_alias2qid, out_f)
print(f"Saved new alias mapping to {new_alias_f}")

result_fpath = os.path.join(errors_dir, 'recall_results.json')
json.save(recall_results, open(result_fpath, 'w'))
json.save(candidate_generator.detailed_metrics, open(os.path.join(errors_dir, "detailed.json"), "w"))
print(f"Recall {recall_results}")
print(f"Metrics {json.dumps(candidate_generator.saved_metrics, indent=4)}")

Processing sentences:   1%|          | 3/357 [00:00<00:14, 23.62it/s]

FILES ['/dfs/scratch0/lorr1/projects/bootleg/data/benchmarks/rss500wikinewdatatest/unfiltered/test_rss500_sen.jsonl', '/dfs/scratch0/lorr1/projects/bootleg/data/benchmarks/rss500wikinewdatatest/unfiltered/test_rss500.jsonl', '/dfs/scratch0/lorr1/projects/bootleg/data/benchmarks/rss500wikinewdatatest/unfiltered/test_rss500_clean.jsonl']
Filtering file /dfs/scratch0/lorr1/projects/bootleg/data/benchmarks/rss500wikinewdatatest/unfiltered/test_rss500_sen.jsonl


0
SANITY CHECK ALL ALIASES:

CHECKING>>> True 1
SPAN CORRECTION:

CHECKING>>> True 1
{'sent_idx_unq': 0, 'aliases': ['aclu'], 'qids': ['Q21637'], 'spans': [[24, 25]], 'sentence': 'The U.S. Patent Office allows genes to be patented as soon as someone isolates the DNA by removing it from the cell , says ACLU attorney Sandra Park .', 'gold': [True]}


1
SANITY CHECK ALL ALIASES:

CHECKING>>> True 1
SPAN CORRECTION:

CHECKING>>> True 1
{'sent_idx_unq': 1, 'aliases': ['aclu'], 'qids': ['Q21637'], 'spans': [[26, 27]], 'sentence': '`` Sup

Processing sentences:   2%|▏         | 7/357 [00:00<00:13, 25.61it/s]

SANITY CHECK ALL ALIASES:

CHECKING>>> True 1
SPAN CORRECTION:

CHECKING>>> True 1


5
SANITY CHECK ALL ALIASES:

CHECKING>>> True 1
SPAN CORRECTION:

CHECKING>>> True 1
{'sent_idx_unq': 5, 'aliases': ['adelson'], 'qids': ['Q319137'], 'spans': [[6, 7]], 'sentence': "Yang even had his daughter take Adelson 's wife , Miriam , shopping when she was in Beijing .", 'gold': [True]}


6
SANITY CHECK ALL ALIASES:

CHECKING>>> True 1
WARNING PREV>>> True True i.i.h.s . ['I.I.H.S']
SPAN CORRECTION:

CHECKING>>> True 1
{'sent_idx_unq': 6, 'aliases': ['i.i.h.s .'], 'qids': ['Q6042346'], 'spans': [[21, 23]], 'sentence': '`` We know more than 10,000 occupants die in frontal crashes each year , `` Adrian Lund , president of the I.I.H.S . , said in a telephone interview .', 'gold': [True]}


7
SANITY CHECK ALL ALIASES:

CHECKING>>> True 1
SPAN CORRECTION:

CHECKING>>> True 1


8
SANITY CHECK ALL ALIASES:

CHECKING>>> True 1
CHECKING>>> True 1
SPAN CORRECTION:

CHECKING>>> True 1
CHECKING>>> True 1
{'s

Processing sentences:   3%|▎         | 11/357 [00:00<00:13, 26.12it/s]

...

Processing sentences:  83%|████████▎ | 298/357 [00:28<00:07,  8.02it/s]



295
SANITY CHECK ALL ALIASES:

CHECKING>>> True 1
SPAN CORRECTION:

CHECKING>>> True 1
{'sent_idx_unq': 295, 'aliases': ['justice department'], 'qids': ['Q1553390'], 'spans': [[6, 8]], 'sentence': 'In response to the lawsuit , Justice Department spokeswoman Tracy Schmaler said , `` We were always willing to work with the committee .', 'gold': [True]}


296
SANITY CHECK ALL ALIASES:

CHECKING>>> True 1
SPAN CORRECTION:

CHECKING>>> True 1


297
SANITY CHECK ALL ALIASES:

CHECKING>>> True 1
CHECKING>>> True 1
SPAN CORRECTION:

CHECKING>>> True 1
CHECKING>>> True 1


298
SANITY CHECK ALL ALIASES:

CHECKING>>> True 1
SPAN CORRECTION:

CHECKING>>> True 1


299
SANITY CHECK ALL ALIASES:

CHECKING>>> True 1
CHECKING>>> True 1
SPAN CORRECTION:

CHECKING>>> True 1
CHECKING>>> True 1
{'sent_idx_unq': 299, 'aliases': ['kayla', 'marblehead'], 'qids': ['Q2358331', 'Q27416'], 'spans': [[0, 1], [5, 6]], 'sentence': 'Kayla , who lives in Marblehead , is the first United States athlete to ever win a 

Processing sentences:  85%|████████▌ | 304/357 [00:28<00:04, 12.49it/s]



300
SANITY CHECK ALL ALIASES:

CHECKING>>> True 1
SPAN CORRECTION:

CHECKING>>> True 1


301
SANITY CHECK ALL ALIASES:

CHECKING>>> True 1
SPAN CORRECTION:

CHECKING>>> True 1
{'sent_idx_unq': 301, 'aliases': ['belgrade lakes'], 'qids': ['Q2301099'], 'spans': [[4, 6]], 'sentence': 'Ken Allen lives in Belgrade Lakes village and can be reached at :', 'gold': [True]}


302
SANITY CHECK ALL ALIASES:

CHECKING>>> True 2
CHECKING>>> True 2
SPAN CORRECTION:

CHECKING>>> True 2
CHECKING>>> True 2
{'sent_idx_unq': 302, 'aliases': ['darrell brock', 'kentucky'], 'qids': ['Q5224589', 'Q7314656'], 'spans': [[35, 37], [44, 45]], 'sentence': 'U of L College Republicans host first annual Legacy Dinner Students , elected officials and candidates gather to discuss politics By : Katherine Smith Larry Cox , state director for Senator Mitch McConnell , and Darrell Brock , chairman of the Republican Party of Kentucky , both attended the first annual Legacy Dinner Republican Party of Kentucky chairman Darr

Processing sentences:  85%|████████▌ | 304/357 [00:40<00:04, 12.49it/s]

In [82]:
candidate_generator.temp

524

In [87]:
entity_dump.get_title("Q21637")

'American Civil Liberties Union'

In [66]:
inv_map = defaultdict(set)
for al in tqdm(alias_map):
    for qid in alias_map[al]:
        inv_map[qid[0]].add(al)


100%|██████████| 9310620/9310620 [00:48<00:00, 193207.11it/s]


In [74]:
for it in candidate_generator.detailed_metrics['not_long_cands_not_sort']:
    print(it["alias"], it["true_qid"], entity_dump._qid2title.get(it["true_qid"], "NAN"))
    print(it["context"])
    print(it["scored_qids"])

mohr davidow ventures -1 NAN
What he 'll invest : : $ 1 million for a plan that spells out the acquisition strategy The Investor : Bill Ericson , general partner , Mohr Davidow Ventures What he 's backed : ParAllele BioScience , Pharmix , Sabrix What he wants now : A device that can identify new types of hospital-borne infections in just a few hours .
[['Q6268969', 137501.0], ['Q770732', 127744.0], ['Q966543', 122501.0], ['Q65049336', 113334.33333333333], ['Q7026503', 111506.0], ['Q959120', 98561.0], ['Q11964053', 97503.0], ['Q7836329', 97501.0], ['Q1356095', 95502.0], ['Q4910213', 94510.0], ['Q7803992', 94001.0], ['Q21062236', 93501.0], ['Q2426152', 93004.0], ['Q5107477', 92509.0], ['Q559436', 92505.0], ['Q442948', 92505.0], ['Q4712033', 92003.0], ['Q68569', 91001.0], ['Q19667774', 89501.0], ['Q698084', 88511.0], ['Q70370', 88505.0], ['Q84863', 88501.0], ['Q492321', 87639.0], ['Q3405534', 86501.0], ['Q872832', 85510.0], ['Q5081015', 85503.0], ['Q15987237', 85503.0], ['Q108030', 85503.

In [78]:
inv_map["Q5386868"]

{'eric', 'eric kearney', 'kearney'}

In [ ]:
save = ["Q45525", "Q1475085", "Q5386868"]

In [14]:
alias = 'bangor'
title = 'bangor civil parish'
1 - nltk.jaccard_distance(set(alias.split()), set(title.split()))

0.33333333333333337

In [15]:
filtered_sentences[0]

{'sentence': 'David and Victoria named their children Brooklyn , Romeo , Cruz , and Harper Seven . ',
 'sent_idx_unq': 0,
 'aliases': ['david_0', 'victoria_0'],
 'spans': [[0, 1], [2, 3]],
 'qids': ['Q10520', 'Q19810'],
 'gold': [True, True],
 'cands': [[['Q8016', 5947],
   ['Q5383', 4466],
   ['Q9916', 3991],
   ['Q192', 2542],
   ['Q134982', 1589],
   ['Q10520', 345904.0],
   ['Q312649', 175136.0],
   ['Q19668637', 175015.0],
   ['Q336640', 165718.0],
   ['Q48373', 165469.0],
   ['Q41564', 165195.0],
   ['Q1176791', 165036.0],
   ['Q1173934', 165003.0],
   ['Q5236705', 163338.33333333334],
   ['Q24248231', 163334.33333333334],
   ['Q1177021', 163334.33333333334],
   ['Q1175688', 162658.0],
   ['Q5236091', 162532.0],
   ['Q2071', 156270.0],
   ['Q178517', 155666.0],
   ['Q214601', 155589.0],
   ['Q184903', 155491.0],
   ['Q353983', 155103.0],
   ['Q1173922', 155081.0],
   ['Q2420499', 155056.0],
   ['Q5239917', 155049.0],
   ['Q338628', 155037.0],
   ['Q1175974', 155013.0],
   ['Q5237

In [16]:
cnt = 0

for sentence in filtered_sentences:
    for i in range(len(sentence['spans'])):
        if sentence['qids'][i] not in [x[0] for x in sentence['cands'][i]]:
            print(cnt)
            print(sentence['sentence'])
            print(sentence['aliases'][i])
            print(sentence['qids'][i], title_map[sentence['qids'][i]])
            print("COMP >>", title_map[sentence['qids'][i]].lower() == " ".join(sentence['sentence'].split()[sentence['spans'][i][0]:sentence['spans'][i][1]]).lower(), title_map[sentence['qids'][i]].lower(), " ".join(sentence['sentence'].split()[sentence['spans'][i][0]:sentence['spans'][i][1]]).lower())
            print(sentence['cands'][i])
            print(' '.join([title_map[x[0]] for x in sentence['cands'][i]]))
            print()
            cnt += 1
        

0
Justin , Stefani , and Kate are among the most popular people on both MTV and Twitter . 
kate_9
Q42493 Katy Perry
COMP >> False katy perry kate
[['Q636', 825], ['Q181484', 718], ['Q34460', 523], ['Q202765', 469], ['Q212531', 442], ['Q229979', 133419.33333333334], ['Q237548', 125157.0], ['Q233946', 125048.0], ['Q19665690', 125002.0], ['Q6375480', 125001.0], ['Q6375798', 123344.33333333333], ['Q29349327', 123342.33333333333], ['Q179414', 115228.0], ['Q723057', 115130.0], ['Q1797489', 115115.0], ['Q3028200', 115077.0], ['Q235084', 115070.0], ['Q231536', 115066.0], ['Q6375783', 115031.0], ['Q16232252', 115017.0], ['Q21063171', 115010.0], ['Q16208419', 115004.0], ['Q46118320', 115003.0], ['Q18705009', 115003.0], ['Q28114718', 115002.0], ['Q259877', 113390.33333333333], ['Q6375462', 113337.33333333333], ['Q6375589', 113336.33333333333], ['Q6375397', 113334.33333333333], ['Q169946', 105262.0]]
Kate Bush Sia (musician) Gwyneth Paltrow Kate Winslet Kate Moss Kate Walsh (actress) Kate Nash Kat

## Debug

In [17]:
q = "Q938005"

In [18]:
title_map['Q12575']

'Ambassador Bridge'

In [19]:
title = 'bangor '
alias = 'bangor'
fuzz.ratio(title, alias)/100

0.92

In [20]:
sentence = 'European authorities fined Google a record $5.1 billion on Wednesday for abusing its power in the mobile phone market and ordered the company to alter its practices'
doc = nlp(sentence)
print([(X.text, X.label_, X.start, X.end) for X in doc.ents])
tokens = sentence.split()
spacy_tokens = [x for x in doc]
cur = 0

for i, x in enumerate(doc):
    print(i,x)
    
print(tokens)

[('European', 'NORP', 0, 1), ('Google', 'ORG', 3, 4), ('$5.1 billion', 'MONEY', 6, 9), ('Wednesday', 'DATE', 10, 11)]
0 European
1 authorities
2 fined
3 Google
4 a
5 record
6 $
7 5.1
8 billion
9 on
10 Wednesday
11 for
12 abusing
13 its
14 power
15 in
16 the
17 mobile
18 phone
19 market
20 and
21 ordered
22 the
23 company
24 to
25 alter
26 its
27 practices
['European', 'authorities', 'fined', 'Google', 'a', 'record', '$5.1', 'billion', 'on', 'Wednesday', 'for', 'abusing', 'its', 'power', 'in', 'the', 'mobile', 'phone', 'market', 'and', 'ordered', 'the', 'company', 'to', 'alter', 'its', 'practices']


In [21]:
sentence = "In two matchups of qualifiers , Sesil Karatantcheva of Kazakhstan cruised past Kiki Bertens of the Netherlands 6-2 , 6-1 , and the Czech Republic 's Andrea Hlavackova defeated Bethanie Mattek-Sands of the United States , 6-3 , 7-5 ."
doc = nlp(sentence)
tokens = sentence.split()
spacy_tokens = [x for x in doc]

spacy_to_split_remap = spacy_to_split_remapping(tokens, spacy_tokens)
print(spacy_to_split_remap)
print([(X.text, X.label_, X.start, X.end, tokens[spacy_to_split_remap[X.start]:spacy_to_split_remap[X.end]]) for X in doc.ents])

for i, x in enumerate(doc):
    print(i,x)
    
print(tokens)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 17, 17, 18, 19, 19, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 30, 30, 31, 32, 33, 34, 35, 36, 36, 36, 37, 38, 38, 38, 39]
[('two', 'CARDINAL', 1, 2, ['two']), ('Sesil Karatantcheva', 'PERSON', 6, 8, ['Sesil', 'Karatantcheva']), ('Kazakhstan', 'GPE', 9, 10, ['Kazakhstan']), ('Kiki Bertens', 'PERSON', 12, 14, ['Kiki', 'Bertens']), ('the Netherlands 6-2', 'FAC', 15, 20, ['the', 'Netherlands', '6-2']), ('6', 'CARDINAL', 21, 22, []), ("the Czech Republic 's", 'GPE', 26, 30, ['the', 'Czech', 'Republic', "'s"]), ('Andrea Hlavackova', 'PERSON', 30, 32, ['Andrea', 'Hlavackova']), ('Bethanie Mattek-Sands', 'PERSON', 33, 37, ['Bethanie', 'Mattek-Sands']), ('the United States', 'GPE', 38, 41, ['the', 'United', 'States']), ('6-3', 'DATE', 42, 45, ['6-3']), ('7', 'CARDINAL', 46, 47, [])]
0 In
1 two
2 matchups
3 of
4 qualifiers
5 ,
6 Sesil
7 Karatantcheva
8 of
9 Kazakhstan
10 cruised
11 past
12 Kiki
13 Bertens
14 of
15 the
16 Nether

In [22]:
spacy_tokens

[In,
 two,
 matchups,
 of,
 qualifiers,
 ,,
 Sesil,
 Karatantcheva,
 of,
 Kazakhstan,
 cruised,
 past,
 Kiki,
 Bertens,
 of,
 the,
 Netherlands,
 6,
 -,
 2,
 ,,
 6,
 -,
 1,
 ,,
 and,
 the,
 Czech,
 Republic,
 's,
 Andrea,
 Hlavackova,
 defeated,
 Bethanie,
 Mattek,
 -,
 Sands,
 of,
 the,
 United,
 States,
 ,,
 6,
 -,
 3,
 ,,
 7,
 -,
 5,
 .]

In [23]:
# RSS

correct = 428

p = correct / 519
r = correct / 520

p, r, 2 * p * r / (p + r)

(0.8246628131021194, 0.823076923076923, 0.8238691049085659)

In [63]:
# KORE
correct = 123

p = correct / 143
r = correct / 144

p, r, 2 * p * r / (p + r)

(0.8601398601398601, 0.8541666666666666, 0.8571428571428572)

In [25]:
cnt = 0

for sentence in filtered_sentences:
    for i in range(len(sentence['spans'])):
        if sentence['qids'][i] not in [x[0] for x in sentence['cands'][i]]:
            print(cnt)
            print(sentence['sentence'])
            print(sentence['aliases'][i])
            print(sentence['qids'][i], title_map[sentence['qids'][i]])
            print("COMP >>", title_map[sentence['qids'][i]].lower() == " ".join(sentence['sentence'].split()[sentence['spans'][i][0]:sentence['spans'][i][1]]).lower(), title_map[sentence['qids'][i]].lower(), " ".join(sentence['sentence'].split()[sentence['spans'][i][0]:sentence['spans'][i][1]]).lower())
            print(sentence['cands'][i])
            print(' '.join([title_map[x[0]] for x in sentence['cands'][i]]))
            print()
            cnt += 1
        

0
Justin , Stefani , and Kate are among the most popular people on both MTV and Twitter . 
kate_9
Q42493 Katy Perry
COMP >> False katy perry kate
[['Q636', 825], ['Q181484', 718], ['Q34460', 523], ['Q202765', 469], ['Q212531', 442], ['Q229979', 133419.33333333334], ['Q237548', 125157.0], ['Q233946', 125048.0], ['Q19665690', 125002.0], ['Q6375480', 125001.0], ['Q6375798', 123344.33333333333], ['Q29349327', 123342.33333333333], ['Q179414', 115228.0], ['Q723057', 115130.0], ['Q1797489', 115115.0], ['Q3028200', 115077.0], ['Q235084', 115070.0], ['Q231536', 115066.0], ['Q6375783', 115031.0], ['Q16232252', 115017.0], ['Q21063171', 115010.0], ['Q16208419', 115004.0], ['Q46118320', 115003.0], ['Q18705009', 115003.0], ['Q28114718', 115002.0], ['Q259877', 113390.33333333333], ['Q6375462', 113337.33333333333], ['Q6375589', 113336.33333333333], ['Q6375397', 113334.33333333333], ['Q169946', 105262.0]]
Kate Bush Sia (musician) Gwyneth Paltrow Kate Winslet Kate Moss Kate Walsh (actress) Kate Nash Kat

## Debug

In [26]:
q = "Q938005"

In [27]:
title_map['Q12575']

'Ambassador Bridge'

In [28]:
title = 'bangor '
alias = 'bangor'
fuzz.ratio(title, alias)/100

0.92

In [29]:
sentence = 'European authorities fined Google a record $5.1 billion on Wednesday for abusing its power in the mobile phone market and ordered the company to alter its practices'
doc = nlp(sentence)
print([(X.text, X.label_, X.start, X.end) for X in doc.ents])
tokens = sentence.split()
spacy_tokens = [x for x in doc]
cur = 0

for i, x in enumerate(doc):
    print(i,x)
    
print(tokens)

[('European', 'NORP', 0, 1), ('Google', 'ORG', 3, 4), ('$5.1 billion', 'MONEY', 6, 9), ('Wednesday', 'DATE', 10, 11)]
0 European
1 authorities
2 fined
3 Google
4 a
5 record
6 $
7 5.1
8 billion
9 on
10 Wednesday
11 for
12 abusing
13 its
14 power
15 in
16 the
17 mobile
18 phone
19 market
20 and
21 ordered
22 the
23 company
24 to
25 alter
26 its
27 practices
['European', 'authorities', 'fined', 'Google', 'a', 'record', '$5.1', 'billion', 'on', 'Wednesday', 'for', 'abusing', 'its', 'power', 'in', 'the', 'mobile', 'phone', 'market', 'and', 'ordered', 'the', 'company', 'to', 'alter', 'its', 'practices']


In [30]:
sentence = "In two matchups of qualifiers , Sesil Karatantcheva of Kazakhstan cruised past Kiki Bertens of the Netherlands 6-2 , 6-1 , and the Czech Republic 's Andrea Hlavackova defeated Bethanie Mattek-Sands of the United States , 6-3 , 7-5 ."
doc = nlp(sentence)
tokens = sentence.split()
spacy_tokens = [x for x in doc]

spacy_to_split_remap = spacy_to_split_remapping(tokens, spacy_tokens)
print(spacy_to_split_remap)
print([(X.text, X.label_, X.start, X.end, tokens[spacy_to_split_remap[X.start]:spacy_to_split_remap[X.end]]) for X in doc.ents])

for i, x in enumerate(doc):
    print(i,x)
    
print(tokens)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 17, 17, 18, 19, 19, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 30, 30, 31, 32, 33, 34, 35, 36, 36, 36, 37, 38, 38, 38, 39]
[('two', 'CARDINAL', 1, 2, ['two']), ('Sesil Karatantcheva', 'PERSON', 6, 8, ['Sesil', 'Karatantcheva']), ('Kazakhstan', 'GPE', 9, 10, ['Kazakhstan']), ('Kiki Bertens', 'PERSON', 12, 14, ['Kiki', 'Bertens']), ('the Netherlands 6-2', 'FAC', 15, 20, ['the', 'Netherlands', '6-2']), ('6', 'CARDINAL', 21, 22, []), ("the Czech Republic 's", 'GPE', 26, 30, ['the', 'Czech', 'Republic', "'s"]), ('Andrea Hlavackova', 'PERSON', 30, 32, ['Andrea', 'Hlavackova']), ('Bethanie Mattek-Sands', 'PERSON', 33, 37, ['Bethanie', 'Mattek-Sands']), ('the United States', 'GPE', 38, 41, ['the', 'United', 'States']), ('6-3', 'DATE', 42, 45, ['6-3']), ('7', 'CARDINAL', 46, 47, [])]
0 In
1 two
2 matchups
3 of
4 qualifiers
5 ,
6 Sesil
7 Karatantcheva
8 of
9 Kazakhstan
10 cruised
11 past
12 Kiki
13 Bertens
14 of
15 the
16 Nether

In [31]:
spacy_tokens

[In,
 two,
 matchups,
 of,
 qualifiers,
 ,,
 Sesil,
 Karatantcheva,
 of,
 Kazakhstan,
 cruised,
 past,
 Kiki,
 Bertens,
 of,
 the,
 Netherlands,
 6,
 -,
 2,
 ,,
 6,
 -,
 1,
 ,,
 and,
 the,
 Czech,
 Republic,
 's,
 Andrea,
 Hlavackova,
 defeated,
 Bethanie,
 Mattek,
 -,
 Sands,
 of,
 the,
 United,
 States,
 ,,
 6,
 -,
 3,
 ,,
 7,
 -,
 5,
 .]

In [32]:
# RSS

correct = 428

p = correct / 519
r = correct / 520

p, r, 2 * p * r / (p + r)

(0.8246628131021194, 0.823076923076923, 0.8238691049085659)

In [33]:
# KORE
correct = 122

p = correct / 143
r = correct / 144

p, r, 2 * p * r / (p + r)

(0.8531468531468531, 0.8472222222222222, 0.8501742160278747)